In [10]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
from kafka import KafkaProducer
import time

# Create a producer object
# Ip can be different in your case
producer = KafkaProducer(bootstrap_servers=['localhost:9092'])
def Convert(a):
    it = iter(a)
    res_dct = dict(zip(it, it))
    return res_dct
df1=pd.DataFrame(columns=['İlan No:', 'İlan Tarihi:', 'Marka:', 'Seri:', 'Model:', 'Yıl:',
       'Yakıt Tipi:', 'Vites Tipi:', 'Motor Hacmi:', 'Motor Gücü:',
       'Kilometre:', 'Boya-değişen:', 'Takasa Uygun:', 'Kimden:'])
while 1==1:
    for page in [1]:
          
      
          url="https://www.arabam.com/ikinci-el/otomobil?page="+str(page)

          r=requests.get(url)
          soup=BeautifulSoup(r.content,'html.parser')
          table=soup.find("table",attrs={"class":"table listing-table w100 border-grey2"})
          cars=table.find_all("a",attrs={"class":"listing-text-new word-break val-middle color-black2018"})
          for car in cars:
            car_link=[]
            car_Link=car.get("href")
            car_Link="https://www.arabam.com/"+car_Link
            car_link.append(car_Link)
            #car_r=requests.get(car_Link)
            soup = BeautifulSoup(requests.get(car_Link).content, 'html.parser')
            table1=soup.find_all("span",attrs={"class":"bli-particle"})
            att=[]

            for i in table1:
                att.append(i.text)
                
               
            data=Convert(att)
            df=pd.DataFrame.from_dict(data, orient="index").T
            df["car_link:"]=car_link
            df1=pd.concat([df1,df],ignore_index=True,join="outer")

    df1=df1.loc[:,['İlan No:', 'İlan Tarihi:', 'Marka:', 'Seri:', 'Model:', 'Yıl:','Yakıt Tipi:', 'Vites Tipi:', 'Motor Hacmi:', 'Motor Gücü:',"Kilometre:",'car_link:']]
    df1.drop_duplicates(subset=['İlan No:'], keep='last',inplace=True)
    df1.to_csv('arabam.csv')



    # Read csv file and quick overview

    
    print(df1.head())


    # DRop null
    df1=df1.dropna()


    # Put columns (including key column) to produce in a single columns named value
    x = df1.to_string(header=False,
                      index=False,
                      index_names=False).split("/n")
    vals = [','.join(ele.split()) for ele in x]

    df1['value'] = vals



    # Iterate through dataframe and produce it
    for index, row in df1.iterrows():
        time.sleep(0.1)
        producer.send('topic1', key=str(row[0]).encode(), value=row[-1].encode())
    df1

    # Observe results from three console consumer under group1

       İlan No:        İlan Tarihi:               Marka:         Seri:  \
0   16224840      18 Aralık 2020           Volkswagen        Passat      
1   16224785      18 Aralık 2020                  BMW      5 Serisi      
2   16223472      18 Aralık 2020                 Fiat          Egea      
3   16223479      18 Aralık 2020              Peugeot           RCZ      
4   16223090      18 Aralık 2020      Mercedes - Benz             C      

                            Model:      Yıl: Yakıt Tipi:        Vites Tipi:  \
0   1.6 TDi BlueMotion Highline      2017       Dizel      Yarı Otomatik      
1    530xd Gran Turismo Premium      2011       Dizel      Yarı Otomatik      
2            1.3 Multijet Urban      2017       Dizel                Düz      
3              1.6 THP Yearling      2011      Benzin      Yarı Otomatik      
4           C 200 d BlueTEC AMG      2016       Dizel           Otomatik      

          Motor Hacmi:       Motor Gücü:      Kilometre:  \
0           1598 cc 

ValueError: Length of values does not match length of index

In [ ]:
vals